# Cluster configuration

### References:
* http://gpdb.docs.pivotal.io/4340/utility_guide/admin_utilities/gpexpand.html
* https://support.pivotal.io/hc/en-us/articles/201202707-gpexpand-example-working-with-one-host-


In [2]:
oldDensity = 12
newDensity = 16

hosts = set([ 'csia4gpw{segment:03d}'.format(segment=i+1) for i in range(70) ])

# This host was excluded from the cluster
hosts = hosts.difference(set(['csia4gpw030']))

In [3]:
numExistingSegments = oldDensity*len(hosts)

print "Currenly", numExistingSegments, "total segments across", len(hosts), "total hosts"
print "Resulting in", 2*oldDensity, "existing segments per host :", 
print oldDensity, "primaries and", oldDensity, "mirrors"

Currenly 828 total segments across 69 total hosts
Resulting in 24 existing segments per host : 12 primaries and 12 mirrors


In [6]:
numAdditionalSegments = (newDensity - oldDensity)*len(hosts)

print "Adding", numAdditionalSegments, "total segments across", len(hosts), "total hosts"
print "Resulting in", 2*(newDensity - oldDensity), "additional segments per host :", 
print (newDensity - oldDensity), "primaries and", (newDensity - oldDensity), "mirrors"

Adding 276 total segments across 69 total hosts
Resulting in 8 additional segments per host : 4 primaries and 4 mirrors


In [7]:
# Add 2, one to skip the master segment, 
# one to exclude last existing segment
dbIbBase = numExistingSegments + 2

# Since content ID starts at zero, this
# calculation works as expected for 
# establishing the base of new ids
contentIdBase = numExistingSegments

# This is actually only primary segments,
# eacho host also takes this number of mirrors also
numNewSegmentsPerHost = (newDensity - oldDensity)

# Twice the number of new segments to account for mirrors
segmentDbIds = set(range(dbIbBase, dbIbBase + 2*numAdditionalSegments))

# Content ID's are unique to primary+mirror pairs
contentIds = range(contentIdBase, contentIdBase + numAdditionalSegments)

# Set the port bases to start from, 
# respecting existing allocations
primaryPortBase = 40000 + oldDensity
primaryRepPortBase = primaryPortBase + 1000
mirrorPortBase = 50000 + oldDensity
mirrorRepPortBase = mirrorPortBase + 1000

thesePrimaryPorts = range(primaryPortBase, primaryPortBase+numNewSegmentsPerHost)
thesePrimaryRepPorts = range(primaryRepPortBase, primaryRepPortBase+numNewSegmentsPerHost)
theseMirrorPorts = range(mirrorPortBase, mirrorPortBase+numNewSegmentsPerHost)
theseMirrorRepPorts = range(mirrorRepPortBase, mirrorRepPortBase+numNewSegmentsPerHost)

In [8]:
from random import randrange
import numpy as np
import pandas as pd

fieldLabels = ["Host", "Interface", "Port", "fselocation", "dbid", "Content", "Role", "ReplicationPort"]
sortedAssignments = pd.DataFrame(columns=fieldLabels)

# Pick k unique items from the available set, 
# return the list of k-items and the new avaialble 
# set (excluding the choices)
def getKIndicies(k, availableSet):
    theseChoices = set()
    availableList = list(availableSet)
    while len(theseChoices) < k:
        thisIndex = randrange(len(availableList))
        theseChoices.add(availableList[thisIndex])
      
    return availableSet.difference(theseChoices), theseChoices

# This keeps track of which db-ids have been assigned
assigned = set()

# This will store a collect of primary and mirror
# entries at each index location
assignments = list()

# For each host, generate a collection of new
# primary and mirror segments, respecting the
# relationship between them (content id)
for i in range(len(hosts)):
    # Shorten the varaible name
    nsph = numNewSegmentsPerHost
    
    # Get a collection of random, unique db ids
    # half for primary segments, half for mirro segments
    segmentDbIds, thisAssignment = getKIndicies(2*nsph, segmentDbIds)
    
    # Track that these db ids have been allocated
    assigned = assigned.union(thisAssignment)
    
    # Take a collection of content ids
    # half as many as db ids, since content id
    # will be the same for a pair of db ids
    theseContentIds = contentIds[nsph*i:nsph*i+nsph]
    
    # Store (zip) this collection of segment+primary entires into a tuple
    assignments.append(zip(thisAssignment, theseContentIds*2)) #, 
                           #thesePrimaryPorts + theseMirrorPorts,
                           #thesePrimaryRepPorts + theseMirrorRepPorts))

# This prints a group of entries, including the
# host and interface assignment
def printHostsLines(host, fields, mounts):
    global sortedAssignments
    
    # unpack (unzip) the tuple into respective fields
    dbIds, contentIds = zip(*fields) #, ports, repPorts = zip(*fields)
    
    # Offset primary and mirror main and replication 
    # ports based off of the mount point assignment
    # to avoid overlaping ports on a single interface
    ports = np.sum([[thesePrimaryPorts + theseMirrorPorts], [ mounts*2]], axis=1 )[0]-1
    repPorts = np.sum([[thesePrimaryRepPorts + theseMirrorRepPorts], [ mounts*2]], axis=1 )[0]-1
    
    # Build a string lookup table with
    # formatting specifiers to generate 
    # the fselocation field
    loc = dict(p='/data{}/primary/gp{}', m='/data{}/mirror/gp{}')
    
    # Loop through the collection of entries 
    # and output the syntacically-valid line
    for (d, c, p, rP, r, i, m) in zip(dbIds, contentIds, ports, repPorts, ['p']*2 + ['m']*2, [1, 2]*2, mounts):
        # Ensure main and replication ports
        # do not collide with preivous assignments
        if m in [3, 4]:
            p += 1
            rP += 1
        # do it
        thisTuple = (host, '-'.join([host, str(i)]), p, loc[r].format(m, c), d, c, r, rP)
        #print ':'.join([str(s) for s in list(thisTuple)])
        sortedAssignments = sortedAssignments.append(dict(zip(fieldLabels,thisTuple)), ignore_index=True)

# Make a list from our host set 
# to allow indexing in the loop
hostList = list(hosts)

# This assigns a primary and mirror collection 
# to every host respecting the mirroring constraint 
# of primary/mirror placement
#print ":".join(fieldLabels)
for h in range(len(hosts)/2):
    # Shorten the varaible name
    nsph = numNewSegmentsPerHost
    
    # Assign a collection of primary segments, 
    # alternating mount point assignment between /data1, /data2
    printHostsLines(hostList[h], assignments[h][:nsph], [1, 2]*2)
    # And assign an unrelated collection of mirror segments,
    # alternating mount point assignment between /data3, /data2
    printHostsLines(hostList[h], assignments[len(hosts)-h-1][:nsph], [3, 4]*2)
    
    # Assign a different collection of primary segments
    printHostsLines(hostList[h+len(hosts)/2], reversed(assignments[h][nsph:]), [1, 2]*2)
    # And assign a different collection of unrelated mirror segments
    printHostsLines(hostList[h+len(hosts)/2], reversed(assignments[len(hosts)-h-1][nsph:]), [3, 4]*2)
    

In [9]:
sortedAssignments = sortedAssignments.sort('dbid')
sortedAssignments

,Host,Interface,Port,fselocation,dbid,Content,Role,ReplicationPort
188,csia4gpw042,csia4gpw042-1,40012,/data3/primary/gp1059,830,1059,p,41012
236,csia4gpw056,csia4gpw056-1,40012,/data3/primary/gp1047,831,1047,p,41012
84,csia4gpw069,csia4gpw069-1,40012,/data3/primary/gp1080,833,1080,p,41012
468,csia4gpw021,csia4gpw021-1,40012,/data3/primary/gp984,834,984,p,41012
320,csia4gpw008,csia4gpw008-1,40011,/data1/primary/gp908,835,908,p,41011
308,csia4gpw009,csia4gpw009-1,40012,/data3/primary/gp1024,836,1024,p,41012
96,csia4gpw068,csia4gpw068-1,40011,/data1/primary/gp852,837,852,p,41011
501,csia4gpw023,csia4gpw023-2,40013,/data4/primary/gp977,838,977,p,41013
165,csia4gpw060,csia4gpw060-2,40013,/data4/primary/gp1061,839,1061,p,41013
274,csia4gpw047,csia4gpw047-1,40013,/data1/mirror/gp898,841,898,m,41013


In [14]:
sortedAssignments.to_csv('gpexpandInput.txt', sep=":", header=False, index=False, float_format="%d")